In [1]:
!pip install pennylane
!pip install pykan
#!git clone https://github.com/pop756/Quantum_machine.git
#%cd Quantum_machine

  Obtaining dependency information for pennylane from https://files.pythonhosted.org/packages/c0/c1/37d745789378cd2d8a48a6e2ecc9deb9948b984c074b87daa721ac0f6cf8/PennyLane-0.36.0-py3-none-any.whl.metadata
  Obtaining dependency information for rustworkx from https://files.pythonhosted.org/packages/49/2a/caeba6c6f0ce37f96dadf329d1d06db7c48cd9dea065df5543a2cdf3b776/rustworkx-0.14.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for autograd from https://files.pythonhosted.org/packages/81/70/d5c7c2a458b8be96495c8b1634c2155beab58cbe864b7a9a5c06c2e52520/autograd-1.6.2-py3-none-any.whl.metadata
  Obtaining dependency information for semantic-version>=2.7 from https://files.pythonhosted.org/packages/6a/23/8146aad7d88f4fcb3a6218f41a60f6c2d4e3a72de72da1825dc7c8f7877c/semantic_version-2.10.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for autoray>=0.6.1 from https://files.pythonhosted.org/packages/e4/b4/51bab53d8b74f4c5bc3b3b8e6f93e3f5ad32e6f3475dbc90fa

In [ ]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import sys
import copy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
import pandas as pd
import pickle
from functions import training

torch.manual_seed(42)

# 하이퍼파라미터 설정
batch_size = 64
epochs = 10
lr = 0.01
PCA_dim = 8
CLS_num = 2
device = 'cpu'


with open('./data/data.pkl','rb') as file:
    data = pickle.load(file)
X = data['X']
y = data['Y']



x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



def Fit_to_quantum(X,PCA_dim):
    pca = PCA(n_components=PCA_dim)
    X_pca = pca.fit_transform(X)
    return X_pca
    



# PyTorch Tensor로 변환
x_train_pca, y_train = torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
x_test_pca, y_test = torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)


class Feature_data_loader(Dataset):
    def __init__(self,x_train,y_train):
        self.feature1 = x_train
        temp = copy.deepcopy(x_train)
        shuffle = torch.randperm(len(temp))
        self.feature2 = temp[shuffle]
        self.y1 = y_train
        temp_y = copy.deepcopy(y_train)
        self.y2 = temp_y[shuffle]
        
    def __len__(self):
        return len(self.feature1)
    def __getitem__(self,idx):
        input1 = self.feature1[idx]
        input2 = self.feature2[idx]
        if self.y1[idx] == self.y2[idx]:
            label = torch.tensor(1.).float()
        else:
            label = torch.tensor(0.).float()
        return [input1,input2],label


# DataLoader 생성


feature_loader = DataLoader(Feature_data_loader(x_train_pca, y_train.float()),batch_size=batch_size,shuffle=True)
test_feature_loader = DataLoader(Feature_data_loader(x_test_pca, y_test.float()),batch_size=batch_size,shuffle=False)
train_loader = DataLoader(TensorDataset(x_train_pca, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test_pca, y_test), batch_size=batch_size, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as np
import math
lamb_list = [0.005*(i+1) for i in range(100)]
lamb_entropy_list = [0.2*(i+1) for i in range(10)]
grid=1

result_dict  = []


# 데이터 로드 및 전처리
"""
data = load_breast_cancer()
X = data.data
y = data.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=PCA_dim)
X_pca = pca.fit_transform(X_scaled)
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.PCA_dim, random_state=seed)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)"""

# Pennylane 장치 설정
dev = qml.device("default.qubit", wires=PCA_dim)


def ZZFeatureMapLayer(features, wires):
    """사용자 정의 ZZFeatureMap 레이어"""
    index = 0
    for i in wires:
        qml.Hadamard(wires=i)
        qml.RZ(features[:,index], wires=i)
        index += 1

    for j in range(0, len(wires)-1):
        qml.CNOT(wires=[j, j+1])
        qml.RZ((features[:,index]), wires=j+1)
        qml.CNOT(wires=[j, j+1])
        index+=1

def ansatz(params):
    for j in range(len(params)):
        # 각 큐비트에 대해 RX, RY, RZ 회전 적용
        for i in range(len(params[0])):
            qml.RY(params[j, i, 0], wires=i)
            qml.RZ(params[j, i, 1], wires=i)
            
        # 인접한 큐비트 간 CNOT 게이트로 엔탱글링
        if j == len(params)-1:
            pass
        else:
            for i in range(len(params[0])-1):
                qml.CNOT(wires=[i, i+1])


# 양자 레이어 정의
@qml.qnode(dev, interface='torch')
def QuantumLayer(features,params):
    ZZFeatureMapLayer(features, wires=range(PCA_dim))
    ansatz(params)
    return qml.probs(wires=range(math.ceil(math.log2(CLS_num))))


## 양자 커널
@qml.qnode(dev, interface='torch')
def Kernal(features1,features2):
    ZZFeatureMapLayer(features1, wires=range(PCA_dim))
    qml.adjoint(ZZFeatureMapLayer)(features2,wires=range(PCA_dim))
    return qml.probs(wires=range(PCA_dim))


class Feature_model(nn.Module):
    def __init__(self,grid,device='cpu'):
        super(Feature_model,self).__init__()
        KAN_model = KAN([PCA_dim,2*PCA_dim+1,PCA_dim*2-1],grid=grid,device=device)
        self.KAN = KAN_model
        self.Kernal = Kernal
    def forward(self,inputs):
        epsilon = 1e-6
        input1 = inputs[0]
        #input1_copy = input1.clone().detach().requires_grad_(True)
        input2 = inputs[1]
        #input2_copy = input2.clone().detach().requires_grad_(True)
        input1 = nn.Sigmoid()(self.KAN(input1))*np.pi
        #input1 = torch.concat([input1,input1_copy],dim=1)
        input2 = nn.Sigmoid()(self.KAN(input2))*np.pi
        #input2 = torch.concat([input2,input2_copy],dim=1)
        output = self.Kernal(input1,input2)
        output = output.type(torch.float32)
        
        return output[:,0].clamp(min=epsilon, max=1-epsilon)
        
    



# 하이브리드 모델 정의

    
def search(params):
    print(f"Test set \n lamb : {params['lamb']} \n lamb_enp :  {params['lamb_entropy']} \n grid : {params['grid']}")
    grid = params['grid']
    feature_model = Feature_model(grid,device); criterion = nn.BCELoss()


    optimizer = optim.Adam(feature_model.parameters(), lr=0.01)

    
    
    # 모델 학습 및 평가
    train_process = training.Early_stop_train_KAN(feature_model, optimizer, criterion)
    train_process.train_model(feature_loader,test_feature_loader,epochs=50,res=15,lamb=params['lamb'],lamb_entropy=params['lamb_entropy'])
    #feature_model.KAN.plot(beta=3,scale=2)
    feature_model.KAN = feature_model.KAN.prune()
    res,pretrain_acc = train_process.test(test_feature_loader)
    print(f"\n Pretrain acc : {pretrain_acc}")
    class HybridModel(nn.Module):
        def __init__(self):
            super(HybridModel, self).__init__()
            self.KAN = feature_model.KAN
            
            self.quantum_layer = QuantumLayer
            self.Q_params = nn.Parameter((torch.rand([PCA_dim,PCA_dim,2])*2-1)*np.pi,requires_grad=True)
        def forward(self, x):
            x = nn.Sigmoid()(self.KAN(x))*np.pi
            quantum_output = self.quantum_layer(x,self.Q_params)
            quantum_output = quantum_output.type(torch.float32)
            return torch.log(quantum_output)
    model = HybridModel(); criterion = nn.NLLLoss()

    for param in model.KAN.parameters():
        param.requires_grad = False
        
    optimizer = optim.Adam([model.Q_params], lr=0.01)
    print("\n\nTest start\n\n")
    train_process = Early_stop_train(model, optimizer, criterion)
    train_process.train_model(train_loader,test_loader,epochs=25,lamb=0)

    _,acc = train_process.test(test_loader)
    print(f"Test Accuracy: {acc:.2f}")
    result_dict.append({"lamb":params['lamb'],"lamb_entropy":f"{params['lamb_entropy']:.1f}", "grid":grid,"acc" : acc,"pretrain_acc" : pretrain_acc})
    result = pd.DataFrame(result_dict)
    result.to_csv('./Results.csv',index=False)
    return -res


SyntaxError: invalid syntax (operation.py, line 1471)

In [ ]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

space = {
    'lamb': hp.uniform('lamb',0,0.1),
    'lamb_entropy': hp.uniform('lamb_entropy',0,2),
    'grid' : hp.choice('grid',range(1,9))
}

best = fmin(fn=search, space=space, algo=tpe.suggest, max_evals=100,verbose=0)

ModuleNotFoundError: No module named 'hyperopt'